Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 予測パイプライン - 自動化 ML

---

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/manymodels/03_Forecasting/03_Forecasting_Pipeline.png)

このノートブックでは、11,973 の AutoML モデルのバッチ予測を行うためのパイプラインを作成します。これらのモデルのトレーニングとスコアリングは、このリポジトリのトレーニング ノートブックで完了しました。ここでは目的の予測期間を基に、予測用のパイプラインを設定します。プロセスを並列化するために、AutoMLPipelineBuilder を活用します。データとモデルの詳細については、データ準備およびトレーニングノートブックを参照してください。

パイプラインの設定は、このリポジトリのトレーニング パイプラインと似ています。手順と機能の詳細については、トレーニングフォルダを参照してください。

### 前提条件
この時点で、次の内容が完了している必要があります:

1. [00_Setup_AML_Workspace notebook](../../00_Setup_AML_Workspace.ipynb) を使用して AML ワークスペースが作成作成済みであること
2. [01_Data_Preparation.ipynb](../../01_Data_Preparation.ipynb) を実行してデータセットが作成済みであること
3. [02_AutoML_Training_Pipeline.ipynb](../02_AutoML_Training_Pipeline/02_AutoML_Training_Pipeline.ipynb) を実行してモデルがトレーニング済みであること

azureml.contrib.automl.pipeline.steps パッケージをインストールします

In [ ]:
# !pip install azureml-contrib-automl-pipeline-steps

## 1.0 ワークスペース、データストア、およびコンピューティングの呼び出し

トレーニング パイプライン ノートブックで行ったように、ワークスペースを呼び出す必要があります。また、データストアとコンピューティングクラスターの変数を作成します。

### ワークスペースに接続する

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd

# ワークスペースのセットアップ
ws= Workspace.from_config() 

# ワークスペースの確認
ws.get_details()

# データストアのセットアップ
dstore = ws.get_default_datastore()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Default datastore name'] = dstore.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

### 既存のコンピュート リソースをアタッチする

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

# クラスタの名前を選択
amlcompute_cluster_name = "cpucluster"

found = False
# ワークスペースにこのコンピュート ターゲットが存在しているか確認
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('既存のコンピュート ターゲットがあります。')
    compute = cts[amlcompute_cluster_name]
    
if not found:
    print('新しいコンピュート ターゲットを作成しています...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D16S_V3',
                                                           min_nodes=2,
                                                           max_nodes=20)
    # クラスタの作成
    compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('クラスタの状態を確認中...')
# 最小ノード数と特定のタイムアウトをポーリングできます。
# min_node_count が提供されていない場合は、クラスターのスケール設定が使用されます。
compute.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
# 現在の AmlCompute ステータスの詳細なビューは、get_status() を使用して確認できます。

### 実験のセットアップ

In [ ]:
from azureml.core import Experiment

experiment = Experiment(ws, 'manymodels-forecasting-pipeline')

## 2.0 登録済みの FileDataset を呼び出す
データ準備ノートブックでは、オレンジジュースの推論データをワークスペースに登録しました。10 シリーズのサブレットまたは 11,973 シリーズの完全なデータセットでパイプラインを実行することを選択できます。10シリーズから始めて拡大することをお勧めします。

In [ ]:
from azureml.core.dataset import Dataset

filedst_10_models = Dataset.get_by_name(ws, name='oj_data_small_inference')
filedst_10_models_input = filedst_10_models.as_named_input('forecast_10_models')
 
#filedst_all_models = Dataset.get_by_name(ws, name='oj_data_inference')
#filedst_all_models_input = filedst_all_models.as_named_input('forecast_all_models')

## 3.0 予測パイプラインの構築
Now that the data, models, and compute resources are set up, we can put together a pipeline for forecasting. 
### Set up the environment to run the script
Specify the conda dependencies for your script. This will allow us to install packages and configure the environment.


データ、モデル、およびコンピューティング リソースを設定したので、予測用のパイプラインをまとめることができます。

スクリプトを実行するための環境のセットアップ
スクリプトの conda 依存関係を指定します。これにより、パッケージをインストールして環境を構成できます。

In [ ]:
training_experiment_name = "<training_experiment_name_goes_here>"
training_pipeline_run_id ="<training_pipeline_run_id_goes_here>"

### エントリスクリプトをラップする構成を作成する
AutoMLPipelineBuilder は、多くのモデルの推論ステップを構築するために使用されます。ユースケースに適したワーカーとノードの数を決定する必要があります。 process_count_per_node は、コンピューティングVMのコア数に基づいています。 node_count は、使用するマスターノードの数を決定します。ノード数を増やすと、トレーニングプロセスが高速化されます。

* <b>experiment</b>: 現在の実験。

* <b>inference_data</b>: 推論データセット。

* <b>compute_target</b>: 推論用のコンピュートターゲット。

* <b>node_count</b>: ユーザー スクリプトの実行に使用するコンピューティング ノードの数。3から始めることをお勧めしますが、トレーニング時間が長すぎる場合には node_countを増やします。

* <b>process_count_per_node</b>: ノードあたりのプロセス数。

* <b>run_invocation_timeout</b>: run() メソッドの呼び出しタイムアウト (秒単位)。タイムアウトは、AutoML の実行の最大トレーニング時間 (及び多少のバッファ) 内に設定する必要があります。デフォルトは 60 秒です。

* <b>output_datastore</b>: 推論結果を出力する出力用データストアを指定します。

* <b>train_experiment_name</b>: Many Models がトレーニングされたトレーニング実験名。

* <b>train_run_id</b>: Many Models がトレーニングされたトレーニング実行ID。

* <b>partition_column_names</b>: パーティション列名。

* <b>time_column_name(オプション)</b>: 時系列の場合は時間列名

* <b>target_column_name(オプション)</b>: 推論データセットにターゲット列がある場合にはターゲット列名

<span style="color:red"><b>注: ワークスペースごとに並列実行できる実行の数には制限があり、現在は、ワークスペースごとに並列処理を 1 つの実験あたり最大 320 回に設定することをお勧めします。ユーザーがより多くの並列処理を必要とし、この制限を増やす場合、要求数過多エラー (HTTP 429) が発生する可能性があります。</b></span>

In [ ]:
from azureml.contrib.automl.pipeline.steps import AutoMLPipelineBuilder

partition_column_names = ['Store', 'Brand']

inference_steps = AutoMLPipelineBuilder.get_many_models_batch_inference_steps(experiment=experiment,
                                                                              inference_data=filedst_10_models_input,
                                                                              compute_target=compute,
                                                                              node_count=2,
                                                                              process_count_per_node=8,
                                                                              run_invocation_timeout=300,
                                                                              output_datastore=dstore,
                                                                              train_experiment_name=training_experiment_name,
                                                                              train_run_id=training_pipeline_run_id,
                                                                              partition_column_names=partition_column_names,
                                                                              time_column_name="WeekStarting",
                                                                              target_column_name="Quantity")

## 4.0 予測パイプラインを実行する
作成した実験を使用して、パイプラインの実行を追跡し、出力を確認できます。

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace = ws, steps=inference_steps)
run = experiment.submit(pipeline)

jupyter ノートブックで予測プロセスを監視する場合は、以下のコマンドを実行します。予測中にログをライブストリーミングします。

**注**: このコマンドはコンピューティング VM（旧Notebook VM）では動作しない場合もありますが、ローカル ラップトップでは動作します。

In [ ]:
run.wait_for_completion(show_output=True)

自動MLモデルを利用して予測ができました。

## 5.0 パイプラインの出力
予測パイプラインは、各店舗のオレンジジュースの販売量をブランド別に予測します。パイプラインは、各店舗の予測を含む1つのファイルを返し、その結果をBLOBの forecasting_output コンテナに出力します。 BLOBコンテナーの詳細は、Outputs+logs の下の 'forecasting_output.txt' にリストされます。

次のコードスニペットは：
1. 並列実行ステップで渡された出力フォルダーの内容をダウンロードし
2. 予測が含まれている parallel_run_step.txt ファイルを pandas データフレームとして読み取り
3. 予測の上位10行を表示します

In [ ]:
import pandas as pd
import shutil
import os
import sys 
from scripts.helper import get_forecasting_output

forecasting_results_name = "forecasting_results"
forecasting_output_name = "many_models_inference_output"

forecast_file = get_forecasting_output(run, forecasting_results_name, forecasting_output_name)
df = pd.read_csv(forecast_file, delimiter=" ", header=None)
df.columns = ["Week Starting", "Store", "Brand", "Quantity",  "Advert", "Price" , "Revenue", "Predicted" ]
print("Prediction has ", df.shape[0], " rows. Here the first 10 rows are being displayed.")
df.head(10)

## 6.0 パイプラインの発行とスケジュール (オプション)

### 6.1 パイプラインを発行する

満足できるパイプラインを作成したら、パイプラインを発行して、後からプログラムで呼び出すことができます。パイプラインの発行と呼び出しの詳細については、この[チュートリアル](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-create-machine-learning-pipelines#publish-a-pipeline)を参照してください。

In [ ]:
# published_pipeline = pipeline.publish(name = 'automl_forecast_many_models',
#                                      description = 'forecast many models',
#                                      version = '1',
#                                      continue_on_step_failure = False)

### 6.2 パイプラインのスケジュール実行
また、時間ベースまたは変更ベースのスケジュールで実行するように[パイプラインをスケジュール](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-trigger-published-pipeline)することもできます。これは、毎月、またはデータドリフトなどの別のトリガーに基づいて、モデルを自動的に再トレーニングまたは推論するために使用できます。

In [ ]:
# from azureml.pipeline.core import Schedule, ScheduleRecurrence
    
# forecasting_pipeline_id = published_pipeline.id

# recurrence = ScheduleRecurrence(frequency="Month", interval=1, start_time="2020-01-01T09:00:00")
# recurring_schedule = Schedule.create(ws, name="automl_forecasting_recurring_schedule", 
#                             description="Schedule Forecasting Pipeline to run on the first day of every week",
#                             pipeline_id=forecasting_pipeline_id, 
#                             experiment_name=experiment.name, 
#                             recurrence=recurrence)